In [ ]:
pip install arch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.9/982.9 kB 11.4 MB/s eta 0:00:00


Data Collection

In [ ]:
import yfinance as yf
import pandas as pd

# Download historical data for Nvidia, AMD and intel
nvidia_data = yf.download('NVDA', start='2000-01-01', end='2024-09-20')
AMD_data = yf.download('AMD', start='2000-01-01', end='2024-09-20')
intel_data = yf.download('INTC', start='2000-01-01', end='2024-09-20')

# Add a new column for each dataset to denote the company
nvidia_data['Company'] = 'NVIDIA'
AMD_data['Company'] = 'AMD'
intel_data['Company'] = 'INTEL'

# Concatenate the datasets into a single dataframe
merged_data = pd.concat([nvidia_data, AMD_data, intel_data])

# Checking the merged data
print(merged_data)

# Preview the data
merged_data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2000-01-03   0.098438   0.099219   0.091927   0.097526   0.089446  300912000   
2000-01-04   0.095833   0.096094   0.090104   0.094922   0.087058  300480000   
2000-01-05   0.092188   0.093750   0.090495   0.091797   0.084191  188352000   
2000-01-06   0.091797   0.091797   0.082292   0.085807   0.078698  120480000   
2000-01-07   0.085417   0.088151   0.084115   0.087240   0.080012   71184000   
...               ...        ...        ...        ...        ...        ...   
2024-09-13  19.490000  19.799999  19.490000  19.660000  19.660000   44236300   
2024-09-16  20.030001  21.070000  19.760000  20.910000  20.910000  149337000   
2024-09-17  21.709999  22.580000  21.240000  21.469999  21.469999  196249800   
2024-09-18  21.360001  21.740000  20.719999  20.770000  20.770000  118727900   
2024-09-19  21.280001  21.690001  21.030

,Open,High,Low,Close,Adj Close,Volume,Company
Date,,,,,,,
2000-01-03,0.098438,0.099219,0.091927,0.097526,0.089446,300912000,NVIDIA
2000-01-04,0.095833,0.096094,0.090104,0.094922,0.087058,300480000,NVIDIA
2000-01-05,0.092188,0.093750,0.090495,0.091797,0.084191,188352000,NVIDIA
2000-01-06,0.091797,0.091797,0.082292,0.085807,0.078698,120480000,NVIDIA
2000-01-07,0.085417,0.088151,0.084115,0.087240,0.080012,71184000,NVIDIA


Data Preprocessing

In [ ]:
# Moving Averages ## !!! something is wrong here !!! ##
merged_data['SMA_20'] = merged_data.groupby('Company')['Close'].rolling(window=20).mean().reset_index(0, drop=True)
merged_data['SMA_50'] = merged_data.groupby('Company')['Close'].rolling(window=50).mean().reset_index(0, drop=True)

# Volatility: Rolling Standard Deviation ## !!! something is wrong here !!! ##
merged_data['Volatility_20'] = merged_data.groupby('Company')['Close'].rolling(window=20).std().reset_index(0, drop=True)

# Drop NaN values that were created by rolling functions
merged_data.dropna(inplace=True)

# Sort the data by date
merged_data_sorted = merged_data.sort_values(by='Date')

# Preview the data with new features
print(merged_data_sorted.head())




                 Open       High       Low      Close  Adj Close      Volume  \
Date                                                                           
2000-03-14   0.309245   0.309245   0.22500   0.229427   0.210419  1567776000   
2000-03-14  62.125000  62.812500  58.90625  58.937500  33.490143    56785000   
2000-03-14  26.000000  26.875000  24.00000  24.000000  24.000000     5936200   
2000-03-15   0.245866   0.252214   0.21250   0.220052   0.201820   995616000   
2000-03-15  59.531250  61.562500  59.34375  60.093750  34.147171    48895600   

           Company     SMA_20     SMA_50  Volatility_20  
Date                                                     
2000-03-14  NVIDIA  22.582812  20.228750       2.952497  
2000-03-14   INTEL   0.156958   0.116275       0.052525  
2000-03-14     AMD  57.018750  51.947500       2.322401  
2000-03-15  NVIDIA  22.726562  20.411250       2.979678  
2000-03-15   INTEL   0.162153   0.118726       0.053407  


Price Prediction (Regression Models)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create boolean dummy variables for the 'Company' column
merged_data['Company_NVIDIA'] = (merged_data['Company'] == 'NVIDIA')
merged_data['Company_AMD'] = (merged_data['Company'] == 'AMD')
merged_data['Company_Intel'] = (merged_data['Company'] == 'Intel')

# Use features and boolean dummy variables to predict the Close price
X = merged_data[['SMA_20', 'SMA_50', 'Volatility_20',
                 'Company_NVIDIA', 'Company_AMD', 'Company_Intel']]
y = merged_data['Close']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Train the model with Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 258.96809869720306


In [ ]:
# Create labels for trend: 1 = uptrend, 0 = downtrend
nvidia_data['Trend'] = (nvidia_data['Close'].shift(-1) > nvidia_data['Close']).astype(int)

# Features for trend prediction
X_trend = nvidia_data[['SMA_20', 'SMA_50', 'Volatility_20']]
y_trend = nvidia_data['Trend']

# Train/Test split
X_train_trend, X_test_trend, y_train_trend, y_test_trend = train_test_split(X_trend, y_trend, test_size=0.2, shuffle=False)

# Logistic Regression for trend prediction
from sklearn.linear_model import LogisticRegression

trend_model = LogisticRegression()
trend_model.fit(X_train_trend, y_train_trend)

# Predict trend
trend_pred = trend_model.predict(X_test_trend)

# Accuracy
from sklearn.metrics import accuracy_score
trend_accuracy = accuracy_score(y_test_trend, trend_pred)
print(f'Trend Prediction Accuracy: {trend_accuracy}')


NameError: name 'nvidia_data' is not defined

In [ ]:
from arch import arch_model

# Use the close prices for GARCH
returns = nvidia_data['Close'].pct_change().dropna()

# Fit the GARCH model
garch_model = arch_model(returns, vol='Garch', p=1, q=1)
garch_fitted = garch_model.fit()

# Predict future volatility
volatility_forecast = garch_fitted.forecast(horizon=5)  # 5-day forecast
print(volatility_forecast.variance[-1:])


In [ ]:
import matplotlib.pyplot as plt

# Plot actual vs predicted prices
plt.figure(figsize=(10, 6))
plt.plot(nvidia_data.index[-len(y_test):], y_test, label='Actual Prices')
plt.plot(nvidia_data.index[-len(y_test):], y_pred, label='Predicted Prices', linestyle='--')
plt.legend()
plt.title('Actual vs Predicted Stock Prices')
plt.show()
